In [12]:
import os
import logging, timeit
#from btEngine2.DataLoader import DataLoader
from btEngine2.MarketData import MarketData
from btEngine2.TradingRule import TradingRule


import platform
import pandas as pd

pd.options.display.float_format = lambda x: f'{x:,.0f}' if abs(x) >= 1000 else (f'{x:.2f}' if abs(x) < 10 else f'{x:.1f}')
# Detect operating system
if platform.system() == "Windows":
    ticker_csv_path = r'G:\Projects\BackTesting1.0\Data\Inputs\TickerList-Futs.csv'
    save_directory = r"G:\Projects\BackTesting1.0\Data\Bloomberg\Futures"
    helper_directory = r'G:\Projects\BackTesting1.0\Data\Bloomberg\HelperFiles'
    bt_folder = r'BackTests\bo_resrch_ml'
    av_folder = r'G:\Projects\BackTesting1.0\Data\Inputs\AssetSizing-Futs.csv'
else:  # Assume macOS for other cases
    ticker_csv_path = r'Data/Inputs/TickerList-Futs.csv'
    save_directory = r"Data/Bloomberg/Futures"
    helper_directory = r'Data/Bloomberg/HelperFiles'
    bt_folder = r'BackTests/bo_resrch_ml'
    av_folder = r'Data/Inputs/AssetSizing-Futs.csv'



# Define paths to auxiliary data for MarketData
tick_values_path = os.path.join(helper_directory, 'fut_val_pt.parquet')
fx_rates_path = os.path.join(helper_directory, 'fxHist.parquet')

# Initialize the MarketData
market_data = MarketData(
    base_directory=save_directory,
    tick_values_path=tick_values_path,
    fx_rates_path=fx_rates_path,
    instrument_type="Futures",
    n_threads=8,  # Number of threads for parallel data loading
    log_level=logging.ERROR  # Set to DEBUG for more detailed logs
)


In [13]:
tick = 'ASD3 Index'
# Access data for a specific ticker
try:
    test_df = market_data.get_ticker_data(tick)
    print(test_df)
except ValueError as e:
    print(e)

# Access all preprocessed data
all_data = market_data.get_data()
print(f"Total tickers loaded: {len(all_data)}")

# Access FX rates
fx_rates = market_data.get_fx_rates()
# Access tick values
tick_values = market_data.get_tick_values()
# Access asset classes
asset_classes = market_data.get_asset_classes()

test_df.to_pandas().to_clipboard()
#market_data = market_data.date_filter(start_date='01012010')

shape: (2_218, 14)
┌────────────┬───────┬───────┬───────┬───┬─────────┬─────────┬─────────────────┬────────────────┐
│ Date       ┆ Open  ┆ High  ┆ Low   ┆ … ┆ BadOHLC ┆ FX_Rate ┆ Tick_Value_Base ┆ Tick_Value_USD │
│ ---        ┆ ---   ┆ ---   ┆ ---   ┆   ┆ ---     ┆ ---     ┆ ---             ┆ ---            │
│ date       ┆ f64   ┆ f64   ┆ f64   ┆   ┆ bool    ┆ f64     ┆ f64             ┆ f64            │
╞════════════╪═══════╪═══════╪═══════╪═══╪═════════╪═════════╪═════════════════╪════════════════╡
│ 2016-01-04 ┆ 49.95 ┆ 49.95 ┆ 49.8  ┆ … ┆ false   ┆ 1.0     ┆ 250.0           ┆ 250.0          │
│ 2016-01-05 ┆ 50.5  ┆ 50.5  ┆ 50.5  ┆ … ┆ true    ┆ 1.0     ┆ 250.0           ┆ 250.0          │
│ 2016-01-06 ┆ 50.15 ┆ 50.15 ┆ 50.15 ┆ … ┆ false   ┆ 1.0     ┆ 250.0           ┆ 250.0          │
│ 2016-01-07 ┆ 49.9  ┆ 49.9  ┆ 49.65 ┆ … ┆ false   ┆ 1.0     ┆ 250.0           ┆ 250.0          │
│ 2016-01-08 ┆ 49.9  ┆ 49.9  ┆ 49.65 ┆ … ┆ false   ┆ 1.0     ┆ 250.0           ┆ 250.0          │
│

In [14]:

from btEngine2.Rules.Momentum.sbo_long import *
from btEngine2.Rules.Momentum.sbo_short import *



rsi_l = dict(rsi_param=(7, 90), N=8)
rsi_l_qe = dict(rsi_param=(7, 90), N=8,exit_quick_rule=True)
rsi_l_qed = dict(rsi_param=(7, 90), N=8, N_min=2, exit_quick_rule=True)
rsi_l_ml_l = dict(rsi_param=(7, 90), N=8, min_dps=5, max_dps=30, probability_threshold=0.5, trend_filter=(100, 'ema'),exit_quick_rule=True)
rsi_l_ml_h = dict(rsi_param=(7, 90), N=8, min_dps=5, max_dps=30, probability_threshold=0.75, trend_filter=(100, 'ema'),exit_quick_rule=True)
rsi_l_ml_fade_h = dict(rsi_param=(7, 90), N=4, min_dps=5, max_dps=30, probability_threshold=-0.1)
rsi_l_ml_fade_m = dict(rsi_param=(7, 90), N=4, min_dps=5, max_dps=30, probability_threshold=-0.2)
rsi_l_ml_fade_l = dict(rsi_param=(7, 90), N=4, min_dps=5, max_dps=30, probability_threshold=-0.4)


pSizeParamsUse = {
    'AssetVol': av_folder,  # Target asset volatility in USD
    'VolLookBack': 30,
    'VolMethod': 'ewm'  # Lookback period for volatility calculation
}

In [15]:
def get_variable_name(var, globals_dict):
    return [name for name in globals_dict if globals_dict[name] is var]


In [16]:

trading_rules_dict = {}

# Define the trading rule functions and their corresponding parameters
trading_rule_functions = {
    'rsi_l': (bo_rsi_long, [rsi_l, rsi_l_qe, rsi_l_qed])
    #'rsi_l_ml_h': (bo_rsi_long_ml, [rsi_l_ml_l, rsi_l_ml_h, rsi_l_ml_fade_h, rsi_l_ml_fade_m, rsi_l_ml_fade_l])
}




# Loop through each trading rule function and its parameters
for rule_name, (rule_function, params_list) in trading_rule_functions.items():
    for params in params_list:
        print(params)
        print(f'name: {get_variable_name(params, globals())[0]}')
        print('---------------------------------')
        rule_label = get_variable_name(params, globals())[0]
        trading_rules_dict[rule_label] = TradingRule(
            market_data=market_data,
            trading_rule_function=rule_function,
            trading_params=params,
            position_sizing_params=pSizeParamsUse,
            cont_rule=False,
            name_label=rule_label,
            bt_folder=bt_folder
        )

trading_rules_dict


{'rsi_param': (7, 90), 'N': 8}
name: rsi_l
---------------------------------
{'rsi_param': (7, 90), 'N': 8, 'exit_quick_rule': True}
name: rsi_l_qe
---------------------------------
{'rsi_param': (7, 90), 'N': 8, 'N_min': 2, 'exit_quick_rule': True}
name: rsi_l_qed
---------------------------------


{'rsi_l': <btEngine2.TradingRule.TradingRule at 0x3020cf110>,
 'rsi_l_qe': <btEngine2.TradingRule.TradingRule at 0x14e5ce1b0>,
 'rsi_l_qed': <btEngine2.TradingRule.TradingRule at 0x3020cff20>}

In [17]:
perf_dfs = {}
pnl_dfs = {}
for tr in trading_rules_dict.values():
    tr.backtest_all_assets(save=True)
    tmpdf = tr.perf_table(byac=True,metric='sharpe')
    perf_dfs[tr.name_lbl] = tmpdf
    tmpdf = tr.perf_table(byac=True, metric='pnl')
    pnl_dfs[tr.name_lbl] = tmpdf

In [ ]:
pnl_dfs['rsi_l_ml_h']

In [ ]:
import matplotlib.pyplot as plt

# Initialize a dictionary to store the equity plots
equity_plots = {}

# Iterate over each trading rule in the rsi_trading_rules dictionary
for rule_name, trading_rule in trading_rules_dict.items():
    # Plot the equity for the trading rule
    equity_plot = trading_rule.plot_equity(byac=True, excl_ac=['eq-vol', 'fx-em'])
    equity_plots[rule_name] = equity_plot

# Plot all the equity curves in a single figure
plt.figure(figsize=(12, 8))
for rule_name, equity_plot in equity_plots.items():
    plt.plot(equity_plot.index, equity_plot['Total'], label=rule_name)

plt.title('Equity Curves for Trading Rules')
plt.xlabel('Date')
plt.ylabel('Equity')
plt.legend()
plt.show()

In [ ]:
# Initialize an empty DataFrame to store the collated results
collated_df = pd.DataFrame()

# Iterate over the equity_plots dictionary
for key, df in equity_plots.items():
    # Select the 'Total' column and rename it to the key
    total_col = df[['Total']].rename(columns={'Total': key})
    # Concatenate the renamed column to the collated_df
    collated_df = pd.concat([collated_df, total_col], axis=1)

collated_df

In [ ]:
pnls_df = collated_df.diff()

n = 125
vol_target = 10000000
# Calculate the rolling annualized volatility
rolling_vol = pnls_df.rolling(window=n).std() * (252 ** 0.5)

# Initialize a DataFrame to store the scaled PnL values
scaled_pnls_df = pnls_df.copy()

# Scale the PnL columns to match the target volatility
for col in pnls_df.columns:
    # Avoid division by zero or NaN issues by filtering valid volatilities
    valid_vol = rolling_vol[col].ffill()
    # Scale factor to match the target volatility
    scale_factor = vol_target / valid_vol
    # Apply scaling, keeping the existing NaNs from the rolling window calculation
    scaled_pnls_df[col] = pnls_df[col] * scale_factor

scaled_pnls_df


In [ ]:
scaled_df = scaled_pnls_df.cumsum()

scaled_df.plot(title='Scaled PnLs', figsize=(16,9), grid=True)

In [ ]:

scaled_ret = scaled_pnls_df
lb = 1

mean_ret = scaled_ret[-lb*252:].mean()
std_ret = scaled_ret[-lb*252:].std()

annualized_mean_ret = mean_ret * 252
annualized_std_ret = std_ret * (252 ** 0.5)

sharpes = annualized_mean_ret / annualized_std_ret

sharpes

In [ ]:
import matplotlib.pyplot as plt

# Define the rolling window size for 3 years (252 trading days per year)
rolling_window = 252 * 1

# Initialize a dictionary to store the rolling Sharpe ratios
rolling_sharpe_ratios = {}


# Calculate the rolling Sharpe ratio for each strategy
for col in scaled_ret.columns:
    rolling_mean = scaled_ret[col].rolling(window=rolling_window).mean()
    rolling_std = scaled_ret[col].rolling(window=rolling_window).std()
    annualized_mean = rolling_mean * 252
    annualized_std = rolling_std * (252 ** 0.5)
    rolling_sharpe = annualized_mean / annualized_std
    rolling_sharpe_ratios[col] = rolling_sharpe

# Plot the rolling Sharpe ratios
plt.figure(figsize=(14, 10))
for col, rolling_sharpe in rolling_sharpe_ratios.items():
    mov_avg = rolling_sharpe.rolling(window=60).mean()
    plt.plot(mov_avg, label=col)

plt.title('Rolling 3-Year Annualized Sharpe Ratios')
plt.xlabel('Date')
plt.ylabel('Sharpe Ratio')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
scaled_ret.corr()

In [31]:


bo_pf = scaled_pnls_df


In [ ]:
import pandas as pd

# Ensure that 'Date' is the index and in datetime format if not already
pnl_series = bo_pf['rsi_l_ml_h']
pnl_series.index = pd.to_datetime(pnl_series.index)

# Resample to monthly frequency and sum the PnL for each month
monthly_pnl = pnl_series.resample('M').sum()

# Create a DataFrame for monthly PnL
monthly_pnl_df = monthly_pnl.to_frame(name='Skew Long and BO L Pullback').reset_index()
monthly_pnl_df['Year'] = monthly_pnl_df['Date'].dt.year
monthly_pnl_df['Month'] = monthly_pnl_df['Date'].dt.month_name().str[:3]

# Pivot the DataFrame to get the desired format
monthly_pnl_pivot = monthly_pnl_df.pivot(index='Year', columns='Month', values='Skew Long and BO L Pullback')
monthly_pnl_pivot = monthly_pnl_pivot[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

# Calculate the total PnL for each year
monthly_pnl_pivot['Total'] = monthly_pnl_pivot.sum(axis=1)

# Calculate annual Sharpe using daily data
# Group daily data by year for the sum of returns and volatility
annual_returns = pnl_series.resample('Y').sum()  # Total return per year
annual_volatility = pnl_series.resample('Y').std() * (252 ** 0.5)  # Annualized volatility

# Calculate Sharpe ratio and handle cases with zero volatility
annual_sharpe = (annual_returns / annual_volatility).replace([float('inf'), -float('inf')], pd.NA)

# Merge annual Sharpe ratio into the pivot table
monthly_pnl_pivot['Sharpe'] = annual_sharpe.values

monthly_pnl_pivot

In [20]:

readme_path = r'BackTests/skew_resrch/abs_skew_long_4a4bb9_e7b522_skew_long_252/README.txt'

exclude = ['eq-divf', 'eq-vol'] + [x for x in asset_classes if 'comm-' in x] + [x for x in asset_classes if 'fx-' in x] + ['stir'] + ['eq-as'] + ['crypto']

# Initialize TradingRule from README file
skew_long = TradingRule.from_readme(
    readme_path=readme_path,
    market_data=market_data,
    sizing_rf=0.25,
    excl_ac=exclude,
    log_level=logging.ERROR,
    # Include any other necessary arguments
)

readme_path = r'BackTests/skew_resrch/abs_skew_short_6c32b2_e7b522_skew_long_252/README.txt'
skew_short = TradingRule.from_readme(
    readme_path=readme_path,
    market_data=market_data,
    sizing_rf=0.25,
    excl_ac=exclude,
    log_level=logging.ERROR,
    # Include any other necessary arguments
)

readme_path = r'BackTests/skew_resrch/abs_skew_combined_e32ea1_e7b522_skew_ls_252/README.txt'
skew_combined = TradingRule.from_readme(
    readme_path=readme_path,
    market_data=market_data,
    sizing_rf=0.25,
    excl_ac=exclude,
    log_level=logging.ERROR,
    # Include any other necessary arguments
)

In [ ]:
skew_long_pnl = skew_long.plot_equity(byac=True)
skew_short_pnl = skew_short.plot_equity(byac=True)
skew_comb_pnl = skew_combined.plot_equity(byac=True)


In [ ]:
# Select and rename the 'Total' columns
skew_long_total = skew_long_pnl[['Total']].rename(columns={'Total': 'Skew_Long_252'})
skew_short_total = skew_short_pnl[['Total']].rename(columns={'Total': 'Skew_Short_252'})
skew_comb_total = skew_comb_pnl[['Total']].rename(columns={'Total': 'Skew_Comb_252'})

# Concatenate the renamed columns to concat_df
concat_df = pd.concat([bo_pf_cum, skew_long_total, skew_short_total, skew_comb_total], axis=1)

concat_df

In [ ]:
df_pnl = concat_df.diff().dropna()
df_pnl.cumsum().plot(figsize=(12, 6), title='Cumulative PnL')

In [ ]:
# Define your volatility targets
vol_targets = {'fast_pb': 10000000, 'rsi_slow_tf': 10000000, 'Skew_Long_252': 10000000, 'Skew_Short_252': 10000000, 'Skew_Comb_252': 10000000}

# Calculate the rolling 252-period annualized volatility for each column
# Annualized vol = std of daily returns * sqrt(252)
rolling_vol = df_pnl.rolling(window=63).std() * (252 ** 0.5)
display(rolling_vol)
# Create a new dataframe to store scaled PnL values
scaled_df = df_pnl.copy()

# Scale the PnL columns to match the target volatility
for col in df_pnl.columns:
    if col in vol_targets:
        # Avoid division by zero or NaN issues by filtering valid volatilities
        valid_vol = rolling_vol[col].ffill()
        # Scale factor to match the target volatility
        scale_factor = vol_targets[col] / valid_vol 
        # Apply scaling, keeping the existing NaNs from the rolling window calculation
        scaled_df[col] = df_pnl[col] * scale_factor



In [25]:
scaled_pnl = scaled_df.cumsum()

In [ ]:
scaled_pnl.plot()

In [ ]:
rolling_vol

In [ ]:
# Calculate the daily mean returns
daily_mean_returns = df_pnl[-252*10:].mean()

# Calculate the daily standard deviation of returns
daily_std_returns = df_pnl[-252*5:].std()

# Annualize the mean returns and standard deviation
annualized_mean_returns = daily_mean_returns * 252
annualized_std_returns = daily_std_returns * (252 ** 0.5)

# Compute the Sharpe ratio
sharpe_ratios = annualized_mean_returns / annualized_std_returns

sharpe_ratios

In [29]:
scaled_pnl['mom_strats'] = scaled_pnl['fast_pb'] + scaled_pnl['rsi_slow_tf']
scaled_pnl['bo fast/slow + long skew'] = scaled_pnl['mom_strats'] + scaled_pnl['Skew_Long_252']
scaled_pnl['bo fast/slow + comb skew'] = scaled_pnl['mom_strats'] + scaled_pnl['Skew_Comb_252']
scaled_pnl['bo slow + comb skew'] = scaled_pnl['rsi_slow_tf'] + scaled_pnl['Skew_Comb_252']
scaled_pnl['bo slow + long skew'] = scaled_pnl['rsi_slow_tf'] + scaled_pnl['Skew_Long_252']

In [ ]:
scaled_pnl.plot(figsize=(16,9))

In [31]:
scaled_ret = scaled_pnl.diff().dropna()


In [ ]:
lb = 15

scaledf = scaled_ret[-lb*252:]

mean_ret = scaledf.mean()
std_ret = scaledf.std()

annualized_mean_ret = mean_ret * 252
annualized_std_ret = std_ret * (252 ** 0.5)

sharpes = annualized_mean_ret / annualized_std_ret

sharpes

In [ ]:
scaled_ret.corr()

In [34]:
df_pnl = scaled_ret

rolling_vol = df_pnl.rolling(window=252).std() * (252 ** 0.5)

# Create a new dataframe to store scaled PnL values
scaled_df = df_pnl.copy()

# Scale the PnL columns to match the target volatility
for col in df_pnl.columns:
    if col in vol_targets:
        # Avoid division by zero or NaN issues by filtering valid volatilities
        valid_vol = rolling_vol[col].ffill()
        # Scale factor to match the target volatility
        scale_factor = 10000000 / valid_vol
        # Apply scaling, keeping the existing NaNs from the rolling window calculation
        scaled_df[col] = df_pnl[col] * scale_factor


In [ ]:
scaled_df.corr()

In [36]:
scaled_df = scaled_df.cumsum()

In [ ]:
scaled_df.iloc[:,-12:].plot(figsize=(16,9))

In [ ]:
lb = 1
scaled_ret = scaled_pnl.diff().dropna()


scaledf = scaled_ret[-lb*252:]

mean_ret = scaledf.mean()
std_ret = scaledf.std()

annualized_mean_ret = mean_ret * 252
annualized_std_ret = std_ret * (252 ** 0.5)

sharpes = annualized_mean_ret / annualized_std_ret

sharpes

In [ ]:
scaled_df

In [ ]:

def calculate_drawdowns(pnl_series):
    """
    Calculate the drawdowns of a PnL series.
    """
    # Calculate the cumulative returns
    cumulative = pnl_series.cumsum()
    # Calculate the running maximum
    running_max = cumulative.cummax()
    # Calculate the drawdown
    drawdown = cumulative - running_max
    # Calculate the duration of the drawdown
    drawdown_duration = (drawdown < 0).astype(int).groupby((drawdown >= 0).astype(int).cumsum()).cumsum()
    
    return drawdown, drawdown_duration

# Initialize dictionaries to store results
worst_drawdowns = {}
average_drawdowns = {}
average_drawdown_durations = {}

# Iterate over each column in scaled_ret
for col in scaled_ret.columns:
    drawdown, drawdown_duration = calculate_drawdowns(scaled_ret[col])
    
    # Store the worst drawdown
    worst_drawdowns[col] = drawdown.min()
    # Store the average drawdown
    average_drawdowns[col] = drawdown[drawdown < 0].mean()
    # Store the average drawdown duration
    average_drawdown_durations[col] = drawdown_duration[drawdown_duration > 0].mean()

# Convert results to DataFrames for better readability
worst_drawdowns_df = pd.DataFrame.from_dict(worst_drawdowns, orient='index', columns=['Worst Drawdown'])
average_drawdowns_df = pd.DataFrame.from_dict(average_drawdowns, orient='index', columns=['Average Drawdown'])
average_drawdown_durations_df = pd.DataFrame.from_dict(average_drawdown_durations, orient='index', columns=['Average Drawdown Duration'])

# Display the results
print("Worst Drawdowns:")
print(worst_drawdowns_df)
print("\nAverage Drawdowns:")
print(average_drawdowns_df)
print("\nAverage Drawdown Durations:")
print(average_drawdown_durations_df)

In [ ]:
worst_drawdowns_df.sort_values(by='Worst Drawdown', ascending=False)

In [ ]:
average_drawdowns_df.sort_values(by='Average Drawdown', ascending=False)

In [ ]:
average_drawdown_durations_df.sort_values(by='Average Drawdown Duration')

In [ ]:
import matplotlib.pyplot as plt

# Pre-defined list of columns to plot
columns_to_plot = ['bo fast/slow + long skew', 'bo fast/slow + comb skew', 'rsi_slow_tf', 'Skew_Long_252']
# Plot histogram
plt.figure(figsize=(12, 8))
for col in columns_to_plot:
    plt.hist(scaled_ret[col], bins=200, alpha=0.5, label=col)


plt.axvline(x=0, color='r', linestyle='--')
plt.xlabel('Daily PnL')
plt.ylabel('Frequency')
plt.title('Histogram of Daily PnL History')
plt.legend()
plt.show()

In [ ]:
import pandas as pd

# Ensure that 'Date' is the index and in datetime format if not already
pnl_series = scaled_ret['bo fast/slow + long skew']
pnl_series.index = pd.to_datetime(pnl_series.index)

# Resample to monthly frequency and sum the PnL for each month
monthly_pnl = pnl_series.resample('M').sum()

# Create a DataFrame for monthly PnL
monthly_pnl_df = monthly_pnl.to_frame(name='Skew Long and BO L Pullback').reset_index()
monthly_pnl_df['Year'] = monthly_pnl_df['Date'].dt.year
monthly_pnl_df['Month'] = monthly_pnl_df['Date'].dt.month_name().str[:3]

# Pivot the DataFrame to get the desired format
monthly_pnl_pivot = monthly_pnl_df.pivot(index='Year', columns='Month', values='Skew Long and BO L Pullback')
monthly_pnl_pivot = monthly_pnl_pivot[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

# Calculate the total PnL for each year
monthly_pnl_pivot['Total'] = monthly_pnl_pivot.sum(axis=1)

# Calculate annual Sharpe using daily data
# Group daily data by year for the sum of returns and volatility
annual_returns = pnl_series.resample('Y').sum()  # Total return per year
annual_volatility = pnl_series.resample('Y').std() * (252 ** 0.5)  # Annualized volatility

# Calculate Sharpe ratio and handle cases with zero volatility
annual_sharpe = (annual_returns / annual_volatility).replace([float('inf'), -float('inf')], pd.NA)

# Merge annual Sharpe ratio into the pivot table
monthly_pnl_pivot['Sharpe'] = annual_sharpe.values

# Add the annual volatility and worst drawdown columns to the pivot table
monthly_pnl_pivot['Annual Volatility'] = annual_volatility.values
# Calculate the worst drawdown for each year
annual_worst_drawdown = pnl_series.resample('Y').apply(lambda x: (x.cumsum() - x.cumsum().cummax()).min())

# Merge annual worst drawdown into the pivot table
monthly_pnl_pivot['Worst Drawdown'] = annual_worst_drawdown.values

monthly_pnl_pivot

In [ ]:
import pandas as pd

# Ensure that 'Date' is the index and in datetime format if not already
pnl_series = scaled_ret['bo slow + long skew']
pnl_series.index = pd.to_datetime(pnl_series.index)

# Resample to monthly frequency and sum the PnL for each month
monthly_pnl = pnl_series.resample('M').sum()

# Create a DataFrame for monthly PnL
monthly_pnl_df = monthly_pnl.to_frame(name='Skew Long and BO L Pullback').reset_index()
monthly_pnl_df['Year'] = monthly_pnl_df['Date'].dt.year
monthly_pnl_df['Month'] = monthly_pnl_df['Date'].dt.month_name().str[:3]

# Pivot the DataFrame to get the desired format
monthly_pnl_pivot = monthly_pnl_df.pivot(index='Year', columns='Month', values='Skew Long and BO L Pullback')
monthly_pnl_pivot = monthly_pnl_pivot[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

# Calculate the total PnL for each year
monthly_pnl_pivot['Total'] = monthly_pnl_pivot.sum(axis=1)

# Calculate annual Sharpe using daily data
# Group daily data by year for the sum of returns and volatility
annual_returns = pnl_series.resample('Y').sum()  # Total return per year
annual_volatility = pnl_series.resample('Y').std() * (252 ** 0.5)  # Annualized volatility

# Calculate Sharpe ratio and handle cases with zero volatility
annual_sharpe = (annual_returns / annual_volatility).replace([float('inf'), -float('inf')], pd.NA)

# Merge annual Sharpe ratio into the pivot table
monthly_pnl_pivot['Sharpe'] = annual_sharpe.values
monthly_pnl_pivot['Annual Volatility'] = annual_volatility.values
annual_worst_drawdown = pnl_series.resample('Y').apply(lambda x: (x.cumsum() - x.cumsum().cummax()).min())

# Merge annual worst drawdown into the pivot table
monthly_pnl_pivot['Worst Drawdown'] = annual_worst_drawdown.values



monthly_pnl_pivot

In [ ]:
import pandas as pd

# Ensure that 'Date' is the index and in datetime format if not already
pnl_series = scaled_ret['bo slow + long skew']
pnl_series.index = pd.to_datetime(pnl_series.index)

# Resample to monthly frequency and sum the PnL for each month
monthly_pnl = pnl_series.resample('M').sum()

# Create a DataFrame for monthly PnL
monthly_pnl_df = monthly_pnl.to_frame(name='bo slow + long skew').reset_index()
monthly_pnl_df['Year'] = monthly_pnl_df['Date'].dt.year
monthly_pnl_df['Month'] = monthly_pnl_df['Date'].dt.month_name().str[:3]

# Pivot the DataFrame to get the desired format
monthly_pnl_pivot = monthly_pnl_df.pivot(index='Year', columns='Month', values='bo slow + long skew')
monthly_pnl_pivot = monthly_pnl_pivot[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

# Calculate the total PnL for each year
monthly_pnl_pivot['Total'] = monthly_pnl_pivot.sum(axis=1)

# Calculate annual Sharpe using daily data
# Group daily data by year for the sum of returns and volatility
annual_returns = pnl_series.resample('Y').sum()  # Total return per year
annual_volatility = pnl_series.resample('Y').std() * (252 ** 0.5)  # Annualized volatility

# Calculate Sharpe ratio and handle cases with zero volatility
annual_sharpe = (annual_returns / annual_volatility).replace([float('inf'), -float('inf')], pd.NA)

# Merge annual Sharpe ratio into the pivot table
monthly_pnl_pivot['Sharpe'] = annual_sharpe.values

monthly_pnl_pivot

In [ ]:
import matplotlib.pyplot as plt

# Define the rolling window size
rolling_window = 125

# Initialize a dictionary to store the rolling Sharpe ratios
rolling_sharpe_ratios = {}

scaled_ret_filt = scaled_ret[['bo slow + long skew', 'bo slow + comb skew', 'bo fast/slow + long skew', 'bo fast/slow + comb skew']]

# Calculate the rolling Sharpe ratio for each strategy
for col in scaled_ret_filt.columns:
    rolling_mean = scaled_ret[col].rolling(window=rolling_window).mean()
    rolling_std = scaled_ret[col].rolling(window=rolling_window).std()
    annualized_mean = rolling_mean * 252
    annualized_std = rolling_std * (252 ** 0.5)
    rolling_sharpe = annualized_mean / annualized_std
    rolling_sharpe_ratios[col] = rolling_sharpe

# Plot the rolling Sharpe ratios
plt.figure(figsize=(14, 10))
for col, rolling_sharpe in rolling_sharpe_ratios.items():
    plt.plot(rolling_sharpe, label=col)

# Calculate the average of the rolling Sharpe ratios for each column
average_rolling_sharpe = {col: rolling_sharpe.mean() for col, rolling_sharpe in rolling_sharpe_ratios.items()}

# Plot horizontal lines for the average rolling Sharpe ratios
for col, avg_sharpe in average_rolling_sharpe.items():
    plt.axhline(y=avg_sharpe, linestyle='--', label=f'Avg {col}: {avg_sharpe:.2f}')


plt.title('Rolling 125-Day Annualized Sharpe Ratios')
plt.xlabel('Date')
plt.ylabel('Sharpe Ratio')
plt.legend()
plt.grid(True)
plt.show()

In [14]:
currtr = trading_rules_dict['rsi_l_ml_h']

In [ ]:
plotdf = currtr.plot_equity(byassets=True, filter_ac=['eq-eu'])

In [ ]:
pnlt = currtr.perf_table(byac=True, metric='pnl')
pnlt

In [23]:
tradeslist = currtr._load_tradeslist()
tradeslist.to_clipboard()

In [ ]:
perft = currtr.perf_table(byac=True, metric='sharpe')
perft

In [25]:
ca1 = currtr._get_btdf('CA1 Index')
ca1.to_clipboard()